In [4]:
# !pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 7.9 MB/s eta 0:00:00


In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.metrics import accuracy_score, mean_absolute_error

In [5]:
import numpy as np
import pandas as pd
import tqdm
import transformers
import string
import re
import nltk
import contractions
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [6]:
raw_data = pd.read_csv('RateMyProfessor_Sample.csv')
print(raw_data)

           professor_name                                 school_name  \
0          Leslie  Looney  University Of Illinois at Urbana-Champaign   
1          Leslie  Looney  University Of Illinois at Urbana-Champaign   
2          Leslie  Looney  University Of Illinois at Urbana-Champaign   
3          Leslie  Looney  University Of Illinois at Urbana-Champaign   
4          Leslie  Looney  University Of Illinois at Urbana-Champaign   
...                   ...                                         ...   
19995  Dennis  Richardson                       Quinnipiac University   
19996  Dennis  Richardson                       Quinnipiac University   
19997  Dennis  Richardson                       Quinnipiac University   
19998  Dennis  Richardson                       Quinnipiac University   
19999  Dennis  Richardson                       Quinnipiac University   

            department_name                    local_name state_name  \
0      Astronomy department   Champaign\xe2\x80\x93

In [10]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

wanted_stopword = ['not', 'no']
for ws in wanted_stopword:
    stop_words.discard(ws)

# Define a function to preprocess comments
def preprocess_text(text):
    text = contractions.fix(str(text))

    # 1. Convert to lowercase
    text = str(text).lower()

    # 2. Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # 3. Tokenize words
    words = word_tokenize(text)

    # 4. Remove stopwords
    words = [word for word in words if word not in stop_words]

    # 5. Perform stemming (optional)
    words = [stemmer.stem(word) for word in words]

    # 6. Join words back into a single string
    processed_text = ' '.join(words)

    return processed_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [11]:
data = raw_data[['student_star', 'student_difficult', 'comments', 'gives_good_feedback', 'caring', 'respected',
             'participation_matters', 'clear_grading_criteria', 'skip_class', 'amazing_lectures',
             'inspirational', 'tough_grader', 'hilarious', 'get_ready_to_read', 'lots_of_homework',
             'accessible_outside_class', 'lecture_heavy', 'extra_credit', 'graded_by_few_things',
             'group_projects','test_heavy', 'so_many_papers', 'beware_of_pop_quizzes', 'IsCourseOnline']]

# Apply preprocessing to the 'comments' column
data = data[data['comments'] != 'No Comments']
data = data[data['comments'] != '']
data['comments'] = data['comments'].apply(preprocess_text)
data = data[data['comments'].apply(len) >= 4]

In [12]:
X = data['comments']
y = data.drop('comments', axis=1)

In [13]:
print("Sample of input data (X):")
for i in range(5):  # Print first 5 comments
    print(f"Comment {i+1}: {X[i][:100]}...")

Sample of input data (X):
Comment 1: class hard twoinon gene knockout content stimul unlik class actual particip pass section easi offer ...
Comment 2: definit go choos prof looney class interest class easi bring note exam dont need rememb lot lot bonu...
Comment 3: overal enjoy class assign straightforward interest didnt enjoy video project felt like no one group ...
Comment 4: ye possibl get definit work content pretti interest tog get super organ class multipl thing due ever...
Comment 5: professor looney great knowledg astronomi explain super easi way elementari class taught class great...


In [14]:
X = data['comments']
y = data[['student_star', 'student_difficult', 'gives_good_feedback', 'caring', 'respected']]

In [15]:
max_words = 10000
max_len = 142 # max comment length

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)
X_pad = pad_sequences(X_seq, maxlen=max_len)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=42)

In [19]:
target_columns = y.columns.tolist()
print(target_columns)

['student_star', 'student_difficult', 'gives_good_feedback', 'caring', 'respected']


In [20]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = max_len

input_layer = Input(shape=(max_len,))
embedding_layer = Embedding(vocab_size, embedding_dim)(input_layer)
lstm_layer1 = LSTM(64, return_sequences=True)(embedding_layer)
lstm_layer2 = LSTM(32)(lstm_layer1)
dense_layer = Dense(64, activation='relu')(lstm_layer2)
dropout_layer = Dropout(0.5)(dense_layer)

outputs = []
for column in target_columns:
    if y[column].dtype == 'float64':
        # For continuous variables (like 'student_star' and 'student_difficult')
        outputs.append(Dense(1, name=column)(dropout_layer))
    else:
        # For binary variables (assuming 0 or 1)
        outputs.append(Dense(1, activation='sigmoid', name=column)(dropout_layer))

model = Model(inputs=input_layer, outputs=outputs)

In [21]:
losses = ['mse' if y[col].dtype == 'float64' else 'binary_crossentropy' for col in target_columns]
metrics = []

for col in target_columns:
    if y[col].dtype == 'float64':
        metrics.append(tf.keras.metrics.MeanAbsoluteError(name=f'{col}_mae'))
    else:
        metrics.append(tf.keras.metrics.BinaryAccuracy(name=f'{col}_accuracy'))

# Compile the model with the updated losses and metrics
model.compile(
    optimizer='adam',
    loss=losses,
    loss_weights=[1.0] * len(target_columns),
    metrics=metrics
)


In [22]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 142)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 142, 142)       │      2,153,146 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 142, 64)        │         52,992 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ (None, 32)             │         12,416 │ lstm[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 64)             │          2,112 │ lstm_1[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 64)             │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ student_star (Dense)      │ (None, 1)              │             65 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ student_difficult (Dense) │ (None, 1)              │             65 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gives_good_feedback       │ (None, 1)              │             65 │ dropout[0][0]          │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ caring (Dense)            │ (None, 1)              │             65 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ respected (Dense)         │ (None, 1)              │             65 │ dropout[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 2,220,991 (8.47 MB)

 Trainable params: 2,220,991 (8.47 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
history = model.fit(
    X_train,
    [y_train[col] for col in y_train.columns],
    epochs=1,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

380/380 ━━━━━━━━━━━━━━━━━━━━ 90s 216ms/step - caring_caring_accuracy: 0.6681 - gives_good_feedback_gives_good_feedback_accuracy: 0.5866 - loss: 9.8306 - respected_respected_accuracy: 0.6862 - student_difficult_student_difficult_mae: 1.4691 - student_star_student_star_mae: 1.7350 - val_caring_caring_accuracy: 0.7152 - val_gives_good_feedback_gives_good_feedback_accuracy: 0.6832 - val_loss: 5.6656 - val_respected_respected_accuracy: 0.7079 - val_student_difficult_student_difficult_mae: 1.0928 - val_student_star_student_star_mae: 1.2888


In [24]:
y_pred = model.predict(X_test)



# Create a list to store the results
results = []

for i, col in enumerate(y_test.columns):
    if y_test[col].dtype == 'float64':
        # For regression tasks, use Mean Absolute Error
        mae = mean_absolute_error(y_test[col], y_pred[i])
        results.append({
            'Output': col,
            'Metric': 'Mean Absolute Error',
            'Value': mae
        })
    else:
        # For binary classification tasks, use Accuracy
        y_pred_binary = (y_pred[i] > 0.5).astype(int)
        accuracy = accuracy_score(y_test[col], y_pred_binary)
        results.append({
            'Output': col,
            'Metric': 'Accuracy',
            'Value': accuracy
        })

# Create the DataFrame from the results list
accuracy_df = pd.DataFrame(results)

# Display the results
print("Model Performance:")
print(accuracy_df.to_string(index=False))

# Calculate and display average accuracy for classification tasks
classification_accuracy = accuracy_df[accuracy_df['Metric'] == 'Accuracy']['Value'].mean()
print(f"\nAverage Classification Accuracy: {classification_accuracy:.4f}")

# Calculate and display average MAE for regression tasks
regression_mae = accuracy_df[accuracy_df['Metric'] == 'Mean Absolute Error']['Value'].mean()
print(f"Average Regression MAE: {regression_mae:.4f}")

119/119 ━━━━━━━━━━━━━━━━━━━━ 12s 85ms/step
Model Performance:
             Output              Metric    Value
       student_star Mean Absolute Error 1.280368
  student_difficult Mean Absolute Error 1.089291
gives_good_feedback            Accuracy 0.685458
             caring            Accuracy 0.727345
          respected            Accuracy 0.706006

Average Classification Accuracy: 0.7063
Average Regression MAE: 1.1848
